In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
columns_to_keep = ["LOCATION", "ISSUE_TIME", "ISSUING_AGENCY_NAME", "VIOLATION_PROCESS_DESC", "ACCIDENT_INDICATOR", "FINE_AMOUNT", "TOTAL_PAID"]

df = pd.read_csv("Moving_Violations_Issued_in_October_2024.csv",
                 usecols=columns_to_keep,
                 dtype={"FINE_AMOUNT": "float64", "TOTAL_PAID": "float64"},
                 low_memory=False)

df = df[df["ACCIDENT_INDICATOR"].isin(["Y", "N"])] #ONLY INCLUDE ROWS WHERE ACCIDENT INDICATOR IS PRESENT


In [ ]:
# Count violations with and without accidents
violation_accident_counts = df.groupby(["VIOLATION_PROCESS_DESC", "ACCIDENT_INDICATOR"]).size().unstack()

# Fill missing values with 0
violation_accident_counts = violation_accident_counts.fillna(0)

# Calculate accident rate for each violation type
violation_accident_counts["Accident Rate (%)"] = (
    violation_accident_counts["Y"] / (violation_accident_counts["Y"] + violation_accident_counts["N"]) * 100
)

# Add a new column to count total violations for each violation type
violation_accident_counts["Total Violations"] = violation_accident_counts["Y"] + violation_accident_counts["N"]

# Filter out violations with a low total count (e.g., fewer than 3 total occurrences)
filtered_violation_accident_counts = violation_accident_counts[violation_accident_counts["Total Violations"] >= 3]

# Sort by highest accident rate, using the filtered version
filtered_violation_accident_counts = filtered_violation_accident_counts.sort_values("Accident Rate (%)", ascending=False)

print(filtered_violation_accident_counts.head(20))


In [ ]:
# Select top violations most associated with accidents (filtered)
top_violations_filtered = filtered_violation_accident_counts.head(15)

plt.figure(figsize=(12, 6))
plt.barh(top_violations_filtered.index, top_violations_filtered["Accident Rate (%)"], color='red')
plt.xlabel("Accident Rate (%)")
plt.ylabel("Violation Type")
plt.title("Top Traffic Violations Most Associated with Accidents in Washington DC Ob")
plt.gca().invert_yaxis()  # Reverse order for readability
plt.show()


In [ ]:
# Filter out these  violations because they are already accidents
excluded_violations = [
"COLLIDING WITH FIXED OBJECT W/NO DAMAGE OR INJURY",
"COLLIDING WITH PEDESTRIAN"
]

filtered_violation_accident_counts = filtered_violation_accident_counts.loc[
    ~filtered_violation_accident_counts.index.isin(excluded_violations)
]

# Print nicely formatted table
print(filtered_violation_accident_counts.head(10).to_string(index=True, float_format="{:.2f}".format))